In [4]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables/alltables2016/internationaltradeinservices2016.xls'
    inputFile = sourceFolder / 'internationaltradeinservices2016.xlsb'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [5]:
next_table = pd.DataFrame()

In [6]:
%%capture
%run "TabA0.ipynb"
next_table = pd.concat([next_table, new_table])

%run "TabB1.ipynb"
next_table = pd.concat([next_table, new_table])

%run "TabB2.ipynb"
next_table = pd.concat([next_table, new_table])

%run "TabB3.ipynb"
next_table = pd.concat([next_table, new_table])

%run "TabC0.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C1 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C1 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C2 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C2 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C3 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C3 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C4 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C4 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C5 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C5 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C6 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C6 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C7 2009-2012.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table C7 2013-2016.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table D1.ipynb"
next_table = pd.concat([next_table, new_table])

%run "Table D2.ipynb"
next_table = pd.concat([next_table, new_table])


In [7]:
next_table.count()

ONS Partner Geography        13532
Year                         13532
Flow                         13532
BOP Service                  13532
International Trade Basis    13532
Measure Type                 13532
Value                        13532
Unit                         13532
dtype: int64

In [8]:
next_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
0,Austria,2012,exports,Total International Trade in Services,BOP,GBP Total,340.404,gbp-million
1,Austria,2013,exports,Total International Trade in Services,BOP,GBP Total,365,gbp-million
2,Austria,2014,exports,Total International Trade in Services,BOP,GBP Total,373.347,gbp-million
3,Austria,2015,exports,Total International Trade in Services,BOP,GBP Total,387.548,gbp-million
4,Austria,2016,exports,Total International Trade in Services,BOP,GBP Total,482,gbp-million


In [9]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
next_table.to_csv(out / 'observations.csv', index = False)

In [10]:
next_table.dtypes

ONS Partner Geography        object
Year                          int32
Flow                         object
BOP Service                  object
International Trade Basis    object
Measure Type                 object
Value                        object
Unit                         object
dtype: object